In [10]:
import time 
import requests
import cv2
import operator
import numpy as np

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognizeinvideo'
_key = 'c0958b1e63134377960c2796e82aa14e'
_maxNumRetries = 10

def processRequest( json, data, headers ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = None )

        if response.status_code == 429: 

            print("Message: %s" % ( response.json()['error']['message'] ))

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print('Error: failed after retrying!')
                break

        elif response.status_code == 200 or response.status_code == 201 or response.status_code == 202:

            print(response.headers)
            
            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print("Error code: %d" % ( response.status_code ))
            print("Message: %s" % ( response.json()['error']['message'] ))

        break
        
    return result

def renderResultOnImage( result, img ):
    
    """Display the obtained results onto the input image"""
    
    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        cv2.rectangle( img,(faceRectangle['left'],faceRectangle['top']),
                           (faceRectangle['left']+faceRectangle['width'], faceRectangle['top'] + faceRectangle['height']),
                       color = (255,0,0), thickness = 5 )


    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        currEmotion = max(currFace['scores'].items(), key=operator.itemgetter(1))[0]


        textToWrite = "%s" % ( currEmotion )
        cv2.putText( img, textToWrite, (faceRectangle['left'],faceRectangle['top']-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1 )

In [2]:
pathToFileInDisk = r'Video.mp4'
with open( pathToFileInDisk, 'rb' ) as f:
    data = f.read()

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None

result = processRequest( json, data, headers )

print(result)

{'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '0', 'Expires': '-1', 'Operation-Location': 'https://westus.api.cognitive.microsoft.com/emotion/v1.0/operations/48e01790-ba1f-40f4-b096-19c8ae42845d', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET, ARR/3.0, ASP.NET', 'apim-request-id': '3186e41f-4c30-482d-a55d-bbc0f9c70921', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'proxy-request-id': 'f27173d8-ac33-488c-a086-81707ae9117b', 'Date': 'Sun, 11 Jun 2017 23:38:39 GMT'}
None


In [14]:
_urlvid="https://westus.api.cognitive.microsoft.com/emotion/v1.0/operations/48e01790-ba1f-40f4-b096-19c8ae42845d"
json= None
headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'
data=None
response = requests.request( 'get', _urlvid, json = json, data = data, headers = headers, params = None )
print(response.content)
data =response.json()
parsed_json = json.loads(data['processingResult'])
fragments=parsed_json['fragments']
for fragment in fragments:
    if 'interval' in fragment:
        interval = fragment['interval']    
        print(interval)

b'{"status":"Succeeded","progress":100.0,"createdDateTime":"2017-06-11T23:38:38.1767595Z","lastActionDateTime":"2017-06-11T23:38:43.1809497Z","processingResult":"{\\"version\\":1,\\"timescale\\":10144,\\"offset\\":0,\\"framerate\\":26.4167,\\"width\\":1472,\\"height\\":482,\\"fragments\\":[{\\"start\\":0,\\"duration\\":15744,\\"interval\\":384,\\"events\\":[[{\\"windowFaceDistribution\\":{\\"neutral\\":1,\\"happiness\\":0,\\"surprise\\":0,\\"sadness\\":0,\\"anger\\":0,\\"disgust\\":0,\\"fear\\":0,\\"contempt\\":0},\\"windowMeanScores\\":{\\"neutral\\":0.825154,\\"happiness\\":0.0717991,\\"surprise\\":0.0770635,\\"sadness\\":0.0163991,\\"anger\\":0.00161595,\\"disgust\\":0.00206743,\\"fear\\":0.00192307,\\"contempt\\":0.00397776}}],[],[],[],[],[],[],[],[],[],[],[],[],[],[{\\"windowFaceDistribution\\":{\\"neutral\\":1,\\"happiness\\":0,\\"surprise\\":0,\\"sadness\\":0,\\"anger\\":0,\\"disgust\\":0,\\"fear\\":0,\\"contempt\\":0},\\"windowMeanScores\\":{\\"neutral\\":0.684172,\\"happiness\

AttributeError: 'NoneType' object has no attribute 'loads'

In [15]:
frameEmotion = dict()
import json
counter = 0
data = responseJson=response.json()
parsed_json = json.loads(data['processingResult'])
timescale=parsed_json['timescale']
framerate=parsed_json['framerate']
fragments=parsed_json['fragments']
for fragment in fragments:
    if 'duration' in fragment:
        duration = fragment['duration']
    if 'start' in fragment:
        start = fragment['start']
    if 'interval' in fragment:
        interval = fragment['interval']
    if 'events' in fragment:
        events = fragment['events']
        frameStart= float(start)/timescale * float(framerate)
        frameInterval = float(interval)/timescale * float(framerate)
        contador=0
        for event in events:
            scores = event[0]
            meanScore = scores['windowMeanScores']
            meanFace = scores['windowFaceDistribution']
            currEmotion = max(meanScore.items(), key=operator.itemgetter(1))[0]
            frameEmotion[int(frameStart+frameInterval*contador)]= currEmotion
            contador=contador+1  

cap = cv2.VideoCapture('/Users/xavierochoa/Downloads/Video2_frag5.mp4')



# Release everything if job is finished


currEmotion="None"
currFrame=0
for i in range(0,1000):
    if currFrame%25==0 :
        newEmotion=frameEmotion[currFrame]
        print(currFrame)
    if newEmotion is not None:
        currEmotion = newEmotion
    ret, frame = cap.read()
    textToWrite = "%s" % ( currEmotion )
    cv2.putText( frame, textToWrite, (800,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1 )
    filename = "video2images/frame" + str(currFrame) +".png"
    cv2.imwrite(filename,frame)
    currFrame=currFrame+1
    
    

cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range

In [32]:
import json
import csv
with open('emotionVideoUno.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    counter = 0
    data = responseJson=response.json()
    parsed_json = json.loads(data['processingResult'])
    timescale=parsed_json['timescale']
    framerate=parsed_json['framerate']
    fragments=parsed_json['fragments']
    prevEmotion="None"
    prevTimeStart=0.0
    prevTimeEnd=0.0
    currTimeStart=0.0
    currTimeEnd=0.0
    currEmotion="None"
    for fragment in fragments:
        if 'duration' in fragment:
            duration = fragment['duration']
        if 'start' in fragment:
            start = fragment['start']
        if 'interval' in fragment:
            interval = fragment['interval']
        if 'events' in fragment:
            events = fragment['events']
            frameStart= float(start)/timescale 
            frameInterval = float(interval)/timescale 
            contador=1
            print(events)
            print("Entro al For")
            for event in events:
                if (len(event)>0):
                    print(event)
                    print(len(event))
                    print("Nuevo evento")
                    scores = event[0]
                    meanScore = scores['windowMeanScores']
                    meanFace = scores['windowFaceDistribution']
                    currEmotion = max(meanScore.items(), key=operator.itemgetter(1))[0]
                    currTimeStart = frameStart+frameInterval*(contador-1)
                    currTimeEnd=frameStart+frameInterval*contador
                    if currEmotion!=prevEmotion:
                        csvwriter.writerow((prevTimeStart, currTimeEnd, prevEmotion))
                        prevEmotion=currEmotion
                        prevTimeStart=currTimeEnd
                contador=contador+1
    csvwriter.writerow((prevTimeStart, currTimeEnd, currEmotion))



[[{'windowFaceDistribution': {'neutral': 1, 'happiness': 0, 'surprise': 0, 'sadness': 0, 'anger': 0, 'disgust': 0, 'fear': 0, 'contempt': 0}, 'windowMeanScores': {'neutral': 0.825154, 'happiness': 0.0717991, 'surprise': 0.0770635, 'sadness': 0.0163991, 'anger': 0.00161595, 'disgust': 0.00206743, 'fear': 0.00192307, 'contempt': 0.00397776}}], [], [], [], [], [], [], [], [], [], [], [], [], [], [{'windowFaceDistribution': {'neutral': 1, 'happiness': 0, 'surprise': 0, 'sadness': 0, 'anger': 0, 'disgust': 0, 'fear': 0, 'contempt': 0}, 'windowMeanScores': {'neutral': 0.684172, 'happiness': 0.303796, 'surprise': 0.005928, 'sadness': 0.00288099, 'anger': 0.000379093, 'disgust': 0.000370278, 'fear': 0.000193526, 'contempt': 0.00228025}}], [], [], [], [], [], [], [], [], [], [], [], [], [{'windowFaceDistribution': {'neutral': 0, 'happiness': 1, 'surprise': 0, 'sadness': 0, 'anger': 0, 'disgust': 0, 'fear': 0, 'contempt': 0}, 'windowMeanScores': {'neutral': 0.000482623, 'happiness': 0.999479, 's